In [10]:
import csv
csv_file = 'staff_diplom.csv'  # Путь к исходному CSV файлу
txt_file = 'output.txt'  # Путь к выходному TXT файлу
with open(csv_file, mode='r', encoding='utf-8') as csv_input, \
     open(txt_file, mode='w', encoding='utf-8') as txt_output:
    csv_reader = csv.reader(csv_input)
    for row in csv_reader:
        s = row[0]
        s = s.split(';')
        txt_output.write(s[2][1:-1] + '\n')